In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import emcee
import corner
import pickle
from astropy import units as u
from astropy import constants as const

%matplotlib widget

In [2]:
var_path = "example data/tau_0.050"

t_flux = np.load(var_path + "_t.npy")[500:2500]
flux = np.load(var_path + "_f.npy")[500:2500]
flux_err = np.load(var_path + "_ferr.npy")[500:2500]

mu = np.mean(flux)
flux = (flux / mu - 1) * 1e3
flux_err = flux_err * 1e3 / mu

t_rad_full = np.load(var_path + "_t.npy")[700:1750]
rv_full = np.load(var_path + "_rv.npy")[700:1750]
rv_err_full = np.load(var_path + "_rverr.npy")[700:1750]

In [3]:
# Pull a random cadence of N~10,25,50
rng_seed = 100 #404
n_rv_cadence = 20

random = np.random.default_rng(rng_seed)
inds = np.sort(random.choice(np.arange(len(t_rad_full)), size=n_rv_cadence, replace=False))

#inds = np.arange(0, len(t_rad_full), 225)

t_rad = t_rad_full[inds]
rv = rv_full[inds]
rv_err = rv_err_full[inds]

trained_data_dic = {"Flux Time":t_flux, "Flux":flux, "Flux Error":flux_err,
                    "RV Time":t_rad_full, "RV":rv_full, "RV Error":rv_err,
                    "Sampled RV Time":t_rad, "Sampled RV":rv, "Sampled RV Error":rv_err,
                    "RNG Seed":rng_seed, "N RV":n_rv_cadence}

In [4]:
def matern52(t1, t2, *, lnsigma = np.log(1.0), lnrho = np.log(3.0)):
    rho = np.exp(lnrho)
    sigma = np.exp(lnsigma)
    x = np.sqrt(5) * np.abs(t1 - t2) / rho
    return sigma ** 2 * (1 + x + x ** 2 / 3) * np.exp(-x)

def matern52_cross(t1, t2, *, lnsigma = np.log(1.0), lnrho = np.log(3.0)):
    rho = np.exp(lnrho)
    sigma = np.exp(lnsigma)
    x = np.sqrt(5) * np.abs(t1 - t2) / rho
    return np.sqrt(5) * sigma ** 2 * np.sign(t1 - t2) * (x + x ** 2) / (3 * rho) * np.exp(-x)

def matern52_grad(t1, t2, *, lnsigma = np.log(1.0), lnrho = np.log(3.0)):
    rho = np.exp(lnrho)
    sigma = np.exp(lnsigma)
    x = np.sqrt(5) * np.abs(t1 - t2) / rho
    return (5 / 3) * sigma ** 2 * (1 + x - x ** 2) / rho ** 2 * np.exp(-x)

def sample_gp(random, K, size=None):
    return random.multivariate_normal(np.zeros(K.shape[0]), K, size=size)

In [5]:
def K_11(t_1, t_2, p):
    if isinstance(t_1, np.ndarray) and isinstance(t_2, np.ndarray):
        t_1 = t_1[:, None]
        t_2 = t_2[None, :]
        
    A, B, C, D, lnrho = p
    
    first_term = A**2 * matern52(t_1, t_2, lnrho=lnrho)
    # Note that the 2nd and 3rd terms cancel by a sign
    fourth_term = B**2 * matern52_grad(t_1, t_2, lnrho=lnrho)
    
    return first_term + fourth_term
    
def K_12(t_1, t_2, p):
    
    if isinstance(t_1, np.ndarray) and isinstance(t_2, np.ndarray):
        t_1 = t_1[:, None]
        t_2 = t_2[None, :]
    
    A, B, C, D, lnrho = p
    
    first_term = A*C * matern52(t_1, t_2, lnrho=lnrho)
    second_term = A*D * matern52_cross(t_1, t_2, lnrho=lnrho)
    third_term = B*C * matern52_cross(t_2, t_1, lnrho=lnrho)
    fourth_term = B*D * matern52_grad(t_1, t_2, lnrho=lnrho)
    
    return first_term + second_term + third_term + fourth_term

def K_21(t_1, t_2, p):
    
    if isinstance(t_1, np.ndarray) and isinstance(t_2, np.ndarray):
        t_1 = t_1[:, None]
        t_2 = t_2[None, :]
    
    A, B, C, D, lnrho = p
    
    first_term = C*A * matern52(t_1, t_2, lnrho=lnrho)
    second_term = C*B * matern52_cross(t_1, t_2, lnrho=lnrho)
    third_term = D*A * matern52_cross(t_2, t_1, lnrho=lnrho)
    fourth_term = D*B * matern52_grad(t_1, t_2, lnrho=lnrho)
    
    return first_term + second_term + third_term + fourth_term
    
def K_22(t_1, t_2, p):
    
    if isinstance(t_1, np.ndarray) and isinstance(t_2, np.ndarray):
        t_1 = t_1[:, None]
        t_2 = t_2[None, :]
    
    A, B, C, D, lnrho = p
    
    first_term = C**2 * matern52(t_1, t_2, lnrho=lnrho)
    # Note the 2nd and 3rd terms cancel by a sign
    fourth_term = D**2 * matern52_grad(t_1, t_2, lnrho=lnrho)
    
    return first_term + fourth_term

In [6]:
def cov_mat(params, t_f, t_rv):
    """
    function to build covariance matrix
    """
    Kappa11 = K_11(t_f, t_f, params)
    Kappa12 = K_12(t_f, t_rv, params)
    Kappa21 = Kappa12.T
    Kappa22 = K_22(t_rv, t_rv, params)

    cov = np.concatenate((
          np.concatenate((Kappa11, Kappa12), axis=1),
          np.concatenate((Kappa21, Kappa22), axis=1),
          ), axis=0)
    
    return cov

In [7]:
def gp_log_like(r, K):
    """
    Pulled from Dan's notebook, updated with Cholesky decomposition
    https://github.com/dfm/gp/blob/main/solutions.ipynb
    
    The multivariate Gaussian ln-likelihood (up to a constant) for the
    vector ``r`` given a covariance matrix ``K``.
    
    :param r: ``(N,)``   The residual vector with ``N`` points.
    :param K: ``(N, N)`` The square (``N x N``) covariance matrix.
    
    :returns lnlike: ``float`` The Gaussian ln-likelihood. 
    """
    # Slow version, factor ~2x slower.
    #return -0.5 * (np.dot(r, np.linalg.solve(K, r)) + np.linalg.slogdet(K)[1])

    # Cholesky decomposition, faster
    # For more info, check out: https://math.stackexchange.com/questions/3158303/using-cholesky-decomposition-to-compute-covariance-matrix-determinant
    try:
        cho_decomp = scipy.linalg.cho_factor(K)
        log_det_cov = 2*np.sum(np.log(np.diag(cho_decomp[0])))
        return -0.5 * (np.dot(r, scipy.linalg.cho_solve(cho_decomp, r)) + log_det_cov) #+ (len(r)*np.log(2.*np.pi)))
    except np.linalg.LinAlgError:
        return -np.inf

In [8]:
def gp_neg_log_prob(params, t_f, t_rv, y, y_err):
    
    jitter = np.exp(params[0])
    kernel_params = params[1:]
    
    # Compute the covariance matrix for the first GP
    K1 = cov_mat(kernel_params[:5], t_f, t_rv)
    
    # Compute the covariance matrix for the second GP
    K2 = cov_mat(kernel_params[5:], t_f, t_rv)
    
    K = K1 + K2
    K[np.diag_indices_from(K)] += y_err**2 + jitter
    
    # Compute the negative log likelihood
    return -gp_log_like(y, K)

In [9]:
def minimize_gp_kernel(y):
    
    p0 = np.array([np.log(0.9**2.),
                   0.5, -0.4, 0.7, 5.0, np.log(2.3),
                   0.5, -0.4, 0.7, 5.0, np.log(2.3)])
    
    result = scipy.optimize.minimize(gp_neg_log_prob, p0, args=(t_flux, t_rad, y, np.concatenate((flux_err, rv_err))))

    return result

In [10]:
y = np.concatenate((flux, rv))
res = minimize_gp_kernel(y)

print(res)

      fun: -2550.555868658847
 hess_inv: array([[ 8.39047356e-04, -4.51509981e-04, -3.52392745e-04,
         1.74827316e-01, -8.44608990e-02,  8.48668381e-04,
         1.20042024e-03, -9.97255202e-04, -8.19239210e-02,
         2.84912577e-01,  1.29100678e-04],
       [-4.51509981e-04,  5.09147942e-04,  1.76920487e-04,
        -1.49355819e-01,  1.92784997e-03, -2.39967242e-04,
        -5.81882689e-04,  7.07855452e-04,  1.49310855e-01,
        -1.28011131e-01, -1.12932771e-04],
       [-3.52392745e-04,  1.76920487e-04,  1.10911427e-03,
        -9.23990053e-02,  9.04553566e-02, -7.45053104e-04,
        -1.09922761e-03,  3.98978841e-05, -4.52113462e-02,
        -2.04478738e-01, -6.04461274e-04],
       [ 1.74827316e-01, -1.49355819e-01, -9.23990053e-02,
         5.79368674e+01, -1.60343639e+00,  1.34959268e-01,
         2.16649869e-01, -2.02394565e-01, -3.75555999e+01,
         4.60416090e+01,  5.52296711e-02],
       [-8.44608990e-02,  1.92784997e-03,  9.04553566e-02,
        -1.60343639e

In [11]:
def cov_mat_test(params, t_test_f, t_test_rv, t_train_f, t_train_rv):
    """
    function to build covariance matrix for test data
    """
    
    #params = params[3:]
    
    Kappa11 = K_11(t_test_f, t_train_f, params)
    Kappa12 = K_12(t_test_f, t_train_rv, params)
    Kappa21 = K_21(t_test_rv, t_train_f, params)
    Kappa22 = K_22(t_test_rv, t_train_rv, params)
            
    cov = np.concatenate((
          np.concatenate((Kappa11, Kappa12), axis=1),
          np.concatenate((Kappa21, Kappa22), axis=1),
          ), axis=0)
    
    return cov

def trained_cov(p):
    """
    function to build covariance matrix from optimized model parameters computed from training set 
    """
    cov_train1 = cov_mat(p[1:6], t_flux, t_rad)
    cov_train2 = cov_mat(p[6:], t_flux, t_rad)
    cov_train = cov_train1 + cov_train2

    cov_train[np.diag_indices_from(cov_train)] += np.concatenate((flux_err, rv_err))**2 + np.exp(p[0])
    
    return cov_train

def predict(p, y, n_test=1000):
    
    cov_train = trained_cov(p)
    
    factor = (scipy.linalg.cholesky(cov_train, overwrite_a=True, lower=False), False)
    alpha  = scipy.linalg.cho_solve(factor, y, overwrite_b=True)
    
    t_test_flux = np.linspace(min(t_flux), max(t_flux), n_test)
    t_test_rv  = np.linspace(min(t_rad_full), max(t_rad_full), n_test)
    
    cov_test_only1 = cov_mat(p[1:6], t_test_flux, t_test_rv)
    cov_test_only2 = cov_mat(p[6:],  t_test_flux, t_test_rv)
    cov_test_only = cov_test_only1 + cov_test_only2
    
    cov_test1 = cov_mat_test(p[1:6], t_test_flux, t_test_rv, t_flux, t_rad) # t_flux and t_rad are training data
    cov_test2 = cov_mat_test(p[6:],  t_test_flux, t_test_rv, t_flux, t_rad)
    cov_test = cov_test1 + cov_test2
    
    mu = np.dot(cov_test, alpha)
    var = cov_test_only[np.diag_indices_from(cov_test_only)]
    inv_cov_test = np.linalg.solve(cov_train, cov_test.T)
    var -= np.sum(cov_test.T * inv_cov_test, axis = 0)
    
    return mu, var, t_test_flux, t_test_rv

In [12]:
mu, var, t_test_flux, t_test_rv = predict(res.x, y)

In [13]:
def plot_data_and_model(p0=None):
    
    fig, ax = plt.subplots(nrows=2, figsize=(12, 6), gridspec_kw={'hspace':0.4})
    
    ax[0].scatter(t_flux, flux, color='black', s=5.0, label='LC w/ errors')#, alpha=0.5)
    ax[0].plot(t_flux, flux-flux_err, color='grey', label='True LC', zorder=9)#, alpha=0.5)
    
    rv_train = ax[1].scatter(t_rad, rv, color='orange', s=100.0, alpha=0.9, zorder=10, marker='*', label='Trained RVs')
    rv_data = ax[1].scatter(t_rad_full, rv_full, color='black', s=5.0, label='RVs w/ errors')#, alpha=0.3)
    true_rv, = ax[1].plot(t_rad_full, rv_full-rv_err_full, color='grey', label='True RVs')#, alpha=0.3)
    
    ax[0].plot(t_test_flux, mu[:len(t_test_flux)], lw=2.0, color='C0', ls='-', label='GP$_\mathrm{LC}$', zorder=10)
    rv_gp, = ax[1].plot(t_test_rv, mu[len(t_test_flux):], lw=2.0, color='C0', ls='-', zorder=1, label='GP$_\mathrm{RV}$')
    
    ax[0].fill_between(t_test_flux, y1=mu[:len(t_test_flux)]-np.sqrt(var[:len(t_test_flux)]),
                       y2=mu[:len(t_test_flux)]+np.sqrt(var[:len(t_test_flux)]), color='C0', alpha=0.5)
    
    ax[1].fill_between(t_test_rv, y1=mu[len(t_test_flux):]-np.sqrt(var[len(t_test_flux):]),
                       y2=mu[len(t_test_flux):]+np.sqrt(var[len(t_test_flux):]), color='C0', alpha=0.25)
    
    ax[0].set_ylabel(r"Norm. Flux (ppt)")
    ax[1].set_ylabel(r"RV (m s$^{-1}$)")
    
    ax[0].set_xlabel(r"Time (day)")
    ax[1].set_xlabel(r"Time (day)")
    
    ax[0].set_xlim([min(t_flux), max(t_flux)])
    ax[1].set_xlim([min(t_rad), max(t_rad)])
    
    ax[0].set_ylim([min(flux)-2.0, max(flux)+2.5])
    ax[1].set_ylim([min(rv_full)-175., max(rv_full)+150.])
    
    ax[0].legend(fontsize=12, markerscale=2.0)
    rv_l1 = ax[1].legend([rv_train, rv_data], ["Trained RVs", "RVs w/ errors"], fontsize=12, markerscale=1.5, loc='lower right')
    rv_l2 = ax[1].legend([rv_gp, true_rv], ["GP$_\mathrm{RV}$", "True RV"], fontsize=12, markerscale=1.5, loc='lower left')
    ax[1].add_artist(rv_l1)

    data_dict_fn = "Two Latent GPs Tests/SqExp/"
    plot_fn = data_dict_fn + 'f500-2500_rv700-1750_nrv{0}_seed{1}.png'.format(n_rv_cadence, rng_seed)
    #plot_fn = data_dict_fn + 'f0-1000_rv2500-3500_nrv7_equal_spaced.png'
    #plt.savefig(plot_fn, bbox_inches='tight', dpi=400)
    #plt.savefig("Plots/two_latent_GP_model_comp_nRV_10.png", bbox_inches='tight', dpi=400)
    
    plt.tight_layout()
    plt.show()
    
#plot_data_and_model(p0 = np.array([0.1, 0.04, 0.07, 5.0, 20.0, 2.0]))
plot_data_and_model()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/var/folders/pp/slx5z51s7j9d_w8l_mb2blvc0001sl/T/ipykernel_21898/3504204243.py:44: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
